# Creating Features Quiz
Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need

import re
# 2) run the cells below to read dataset and build body length feature
# 3) write code to answer the quiz questions 

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

### Read Dataset

In [3]:
stack_overflow_data = 'Train_onetag_small.json'

In [4]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

### Build Body Length Feature

In [5]:
from pyspark.ml.feature import RegexTokenizer, VectorAssembler, Normalizer, StandardScaler
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

In [6]:
regexTokenizer = RegexTokenizer(inputCol="Body", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)

In [7]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn("BodyLength", body_length(df.words))

In [8]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

# Question 1
Select the question with Id = 1112. How many words does its body contain (check the BodyLength column)?

In [9]:
# TODO: write your code to answer question 1

df.select(df['BodyLength']).where(df.Id=='1112').collect()

[Row(BodyLength=63)]

# Question 2
Create a new column that concatenates the question title and body. Apply the same functions we used before to compute the number of words in this combined column. What's the value in this new column for Id = 5123?

In [17]:
from pyspark.sql.functions import col,concat

In [21]:
df = df.withColumn("Combine",concat(col("Title"),col("Body")))

In [22]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [23]:
# TODO: write your code to answer question 2
regexTokenizer_1 = RegexTokenizer(inputCol="Combine", outputCol="TransferedCombine", pattern="\\W")
df = regexTokenizer_1.transform(df)
df = df.withColumn("CombineLength", body_length(df['TransferedCombine']))
df.select(df["CombineLength"]).where(df["Id"]==5123).collect()

[Row(CombineLength=135)]

# Create a Vector
Create a vector from the combined Title + Body length column. In the next few questions, you'll try different normalizer/scaler methods on this new column.

In [24]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [25]:
# TODO: write your code to create this vector
assembler = VectorAssembler(inputCols=["CombineLength"],outputCol="V_Combinelength")

In [26]:
df = assembler.transform(df)

# Question 3
Using the Normalizer method what's the normalized value for question Id = 512?

In [27]:
# TODO: write your code to answer question 3

scaler = Normalizer(inputCol="V_Combinelength", outputCol="Norm_Combinelength")
df = scaler.transform(df)
df.select(df["Norm_Combinelength"]).where(df["Id"]==512).collect()

[Row(Norm_Combinelength=DenseVector([1.0]))]

# Question 4
Using the StandardScaler method (scaling both the mean and the standard deviation) what's the normalized value for question Id = 512?

In [30]:
# TODO: write your code to answer question 4
scaler1 = StandardScaler(inputCol="V_Combinelength", outputCol="Std_CombineLength_1", withStd=False,withMean=False)
scalerModel = scaler1.fit(df)
df = scalerModel.transform(df)
df.select(df["Std_CombineLength_1"]).where(df["Id"]==512).collect()

[Row(Std_CombineLength_1=DenseVector([57.0]))]

# Question 5
Using the MinMAxScaler method what's the normalized value for question Id = 512?

In [31]:
# TODO: write your code to answer question 5
from pyspark.ml.feature import MinMaxScaler
scaler2 = MinMaxScaler(inputCol="V_Combinelength", outputCol="MinMax_CombineLength")

# Compute summary statistics and generate MinMaxScalerModel
scalerModel = scaler2.fit(df)

# rescale each feature to range [min, max].
df = scalerModel.transform(df)
df.select(df["MinMax_CombineLength"]).where(df["Id"]==512).collect()

[Row(MinMax_CombineLength=DenseVector([0.0062]))]